In [1]:
import numpy as np
import pandas as pd
import os
import math

from scipy import stats #Analysis 
from scipy.stats import norm 

import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline

In [2]:

data =  pd.read_csv("../train.csv", parse_dates=['summary_date']) # 애초에 datatime 유형의 데이터를 읽어 올 때부터 형변환하기
data_label =  pd.read_csv("../train_label.csv")


In [4]:
# data + data_label 
# data['DIAG_NM'] = data_label

label_dict =  dict([(i,a) for i,a in zip(data_label['SAMPLE_EMAIL'], data_label['DIAG_NM'])])

# DIAG_NM
data['DIAG_NM'] = data['EMAIL'].map(label_dict)
data

,EMAIL,summary_date,activity_average_met,activity_cal_active,activity_cal_total,activity_class_5min,activity_daily_movement,activity_high,activity_inactive,activity_inactivity_alerts,...,sleep_temperature_deviation,sleep_temperature_trend_deviation,timezone,sleep_total,CONVERT(activity_class_5min USING utf8),CONVERT(activity_met_1min USING utf8),CONVERT(sleep_hr_5min USING utf8),CONVERT(sleep_hypnogram_5min USING utf8),CONVERT(sleep_rmssd_5min USING utf8),DIAG_NM
0,nia+404@rowan.kr,2020-11-27,1.71875,730,2944,...,14346,0,417,0,...,-0.12,99.99,NaN,\r,2/1/1/1/1/1/2/2/1/1/1/1/1/1/2/2/2/3/2/2/2/2/2/...,0.9/0.9/1.4/1.9/1.1/0.9/0.9/1.1/1.3/1/0.9/1.1/...,0/73/73/73/72/71/70/71/71/71/70/70/73/72/74/74...,4/2/4/3/3/1/2/2/2/2/2/2/3/3/3/4/4/3/2/2/2/2/2/...,0/10/10/10/11/11/10/12/18/13/14/12/10/10/18/17...,CN
1,nia+404@rowan.kr,2020-11-28,1.40625,342,2449,...,6352,0,473,0,...,-0.32,99.99,NaN,\r,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/2/2/2/2/2/2/...,1.2/1.1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,69/70/69/69/70/72/71/72/70/69/69/69/68/68/63/6...,2/4/2/2/2/2/3/1/2/2/4/4/2/2/2/2/2/2/2/2/2/2/4/...,23/23/26/24/18/13/15/14/17/20/24/30/23/25/22/1...,CN
2,nia+404@rowan.kr,2020-11-29,1.46875,401,2544,...,7297,0,586,0,...,0.07,99.99,NaN,\r,1/1/1/1/1/1/1/2/1/1/1/1/2/2/2/2/2/1/1/1/1/1/2/...,1.1/1.1/1.2/1.1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,0/74/73/73/74/74/74/71/71/70/70/69/70/68/66/69...,4/2/4/4/1/1/1/4/4/4/4/4/4/4/2/3/4/2/2/4/2/2/2/...,0/11/14/20/13/14/14/16/27/29/27/20/19/19/14/12...,CN
3,nia+404@rowan.kr,2020-11-30,0.34375,27,1850,...,491,0,176,0,...,-0.41,99.99,NaN,\r,2/1/2/2/1/2/1/1/2/1/1/1/1/1/2/1/1/1/1/1/2/2/2/...,0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/...,73/70/71/72/75/75/73/70/70/70/67/63/63/63/63/6...,4/4/4/4/3/3/3/2/4/4/4/2/2/2/2/2/2/2/2/4/2/2/2/...,24/28/19/17/12/10/17/20/23/23/25/31/26/25/34/3...,CN
4,nia+404@rowan.kr,2020-12-01,1.46875,333,2518,...,5861,0,646,0,...,-0.27,99.99,NaN,\r,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/2/2/3/3/2/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,0/0/0/0/0/0/0/0/69/69/71/69/65/66/64/64/65/66/...,4/4/4/4/4/4/4/4/4/4/4/2/2/2/2/3/3/2/4/4/4/2/2/...,0/0/0/0/0/0/0/0/21/22/26/23/19/29/22/17/14/13/...,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,nia+206@rowan.kr,2020-12-22,1.34375,227,2316,...,3863,3,735,2,...,-0.16,99.99,NaN,\r,2/1/1/2/2/1/1/1/1/2/2/3/2/1/1/1/1/1/1/1/1/1/1/...,1.1/1.4/1.2/0.9/1.2/1.1/0.9/0.9/0.9/1/0.9/0.9/...,0/54/54/54/55/56/0/0/55/52/52/53/54/54/56/57/6...,4/4/4/2/2/2/2/4/2/2/2/1/1/1/1/4/2/2/3/3/3/3/3/...,0/35/39/28/26/41/0/0/52/31/27/30/21/22/27/40/3...,CN
9323,nia+206@rowan.kr,2020-12-24,1.34375,249,2351,...,4411,1,780,4,...,-0.09,99.99,NaN,\r,1/1/1/1/1/1/1/1/1/1/2/3/2/2/2/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,68/66/67/67/68/69/69/70/71/71/71/69/72/70/70/7...,4/4/2/2/2/1/1/1/1/1/1/2/2/3/3/2/2/2/2/2/3/4/2/...,26/16/19/18/16/18/19/17/15/16/15/13/11/13/13/1...,CN
9324,nia+206@rowan.kr,2020-12-26,1.53125,570,2682,...,11057,1,518,1,...,0.41,99.99,NaN,\r,1/1/1/1/1/1/2/2/2/2/2/2/1/1/1/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/1.1/0.9/0.9/1.1/0.9/0.9/0.9/0....,0/0/65/65/66/67/69/72/73/72/73/74/73/74/75/75/...,4/4/2/2/2/2/2/2/2/2/2/2/4/4/4/4/2/3/3/3/2/2/4/...,0/0/13/12/12/13/12/12/12/16/13/12/13/17/13/18/...,CN
9325,nia+206@rowan.kr,2020-12-27,1.34375,295,2331,...,5135,4,578,0,...,0.27,99.99,NaN,\r,1/1/1/2/3/2/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/...,1.1/0.9/0.9/0.9/1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/...,0/0/0/64/64/64/64/65/65/66/66/66/66/68/68/68/6...,4/4/4/4/2/2/2/2/2/2/2/2/2/3/3/3/3/2/4/4/2/2/2/...,0/0/0/14/17/18/14/14/17/15/17/12/17/12/13/12/1...,CN


# Preprocessing

#### feature drop
- timezone
- EMAIL
- sleep_temperature_trend_deviation, sleep_is_longest, sleep_total
    - 모든 값이 1, '\r', 99.99 이므로 제거
- sleep_temperature_deviation
    - sleep_temperature_delta 변수와 중복되므로 deviation 변수를 제거
- activity_class_5min, activity_met_1min
    - convert data
- sleep_hr_5min, sleep_hypnogram_5min, sleep_rmssd_5min
    - convert data
- CONVERT(activity_class_5min USING utf8)
- CONVERT(activity_met_1min USING utf8)
- CONVERT(sleep_hr_5min USING utf8)
- CONVERT(sleep_hypnogram_5min USING utf8)
- CONVERT(sleep_rmssd_5min USING utf8)

#### y variable
- DIAG_NM
    - 정상(CN), 경도인지 장애(MCI), 치매(Dem)

In [5]:
# column의 수가 많기 때문에 효율적으로 관리하기 위해서 
# column의 이름들을 묶어서 진행합시다

drop_cols = ['timezone', 'EMAIL',
             'activity_class_5min', 'activity_met_1min', 
             'sleep_hr_5min', 'sleep_hypnogram_5min', 'sleep_rmssd_5min',
             'sleep_is_longest', 'sleep_temperature_trend_deviation', 'sleep_total', 'sleep_temperature_deviation']
convert_cols = data.iloc[:, 60:66].columns
data_convert = data[convert_cols].copy()
data_convert

,CONVERT(activity_class_5min USING utf8),CONVERT(activity_met_1min USING utf8),CONVERT(sleep_hr_5min USING utf8),CONVERT(sleep_hypnogram_5min USING utf8),CONVERT(sleep_rmssd_5min USING utf8),DIAG_NM
0,2/1/1/1/1/1/2/2/1/1/1/1/1/1/2/2/2/3/2/2/2/2/2/...,0.9/0.9/1.4/1.9/1.1/0.9/0.9/1.1/1.3/1/0.9/1.1/...,0/73/73/73/72/71/70/71/71/71/70/70/73/72/74/74...,4/2/4/3/3/1/2/2/2/2/2/2/3/3/3/4/4/3/2/2/2/2/2/...,0/10/10/10/11/11/10/12/18/13/14/12/10/10/18/17...,CN
1,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/2/2/2/2/2/2/...,1.2/1.1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,69/70/69/69/70/72/71/72/70/69/69/69/68/68/63/6...,2/4/2/2/2/2/3/1/2/2/4/4/2/2/2/2/2/2/2/2/2/2/4/...,23/23/26/24/18/13/15/14/17/20/24/30/23/25/22/1...,CN
2,1/1/1/1/1/1/1/2/1/1/1/1/2/2/2/2/2/1/1/1/1/1/2/...,1.1/1.1/1.2/1.1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,0/74/73/73/74/74/74/71/71/70/70/69/70/68/66/69...,4/2/4/4/1/1/1/4/4/4/4/4/4/4/2/3/4/2/2/4/2/2/2/...,0/11/14/20/13/14/14/16/27/29/27/20/19/19/14/12...,CN
3,2/1/2/2/1/2/1/1/2/1/1/1/1/1/2/1/1/1/1/1/2/2/2/...,0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/...,73/70/71/72/75/75/73/70/70/70/67/63/63/63/63/6...,4/4/4/4/3/3/3/2/4/4/4/2/2/2/2/2/2/2/2/4/2/2/2/...,24/28/19/17/12/10/17/20/23/23/25/31/26/25/34/3...,CN
4,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/2/2/3/3/2/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,0/0/0/0/0/0/0/0/69/69/71/69/65/66/64/64/65/66/...,4/4/4/4/4/4/4/4/4/4/4/2/2/2/2/3/3/2/4/4/4/2/2/...,0/0/0/0/0/0/0/0/21/22/26/23/19/29/22/17/14/13/...,CN
...,...,...,...,...,...,...
9322,2/1/1/2/2/1/1/1/1/2/2/3/2/1/1/1/1/1/1/1/1/1/1/...,1.1/1.4/1.2/0.9/1.2/1.1/0.9/0.9/0.9/1/0.9/0.9/...,0/54/54/54/55/56/0/0/55/52/52/53/54/54/56/57/6...,4/4/4/2/2/2/2/4/2/2/2/1/1/1/1/4/2/2/3/3/3/3/3/...,0/35/39/28/26/41/0/0/52/31/27/30/21/22/27/40/3...,CN
9323,1/1/1/1/1/1/1/1/1/1/2/3/2/2/2/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,68/66/67/67/68/69/69/70/71/71/71/69/72/70/70/7...,4/4/2/2/2/1/1/1/1/1/1/2/2/3/3/2/2/2/2/2/3/4/2/...,26/16/19/18/16/18/19/17/15/16/15/13/11/13/13/1...,CN
9324,1/1/1/1/1/1/2/2/2/2/2/2/1/1/1/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/1.1/0.9/0.9/1.1/0.9/0.9/0.9/0....,0/0/65/65/66/67/69/72/73/72/73/74/73/74/75/75/...,4/4/2/2/2/2/2/2/2/2/2/2/4/4/4/4/2/3/3/3/2/2/4/...,0/0/13/12/12/13/12/12/12/16/13/12/13/17/13/18/...,CN
9325,1/1/1/2/3/2/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/...,1.1/0.9/0.9/0.9/1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/...,0/0/0/64/64/64/64/65/65/66/66/66/66/68/68/68/6...,4/4/4/4/2/2/2/2/2/2/2/2/2/3/3/3/3/2/4/4/2/2/2/...,0/0/0/14/17/18/14/14/17/15/17/12/17/12/13/12/1...,CN


In [6]:
data_convert.iat[0, 2].count('/')

119

### data_convert '/' 제거, label Encoding

#### - CONVERT(activity_class_5min USING utf8)

In [7]:
signal_list = list(data_convert['CONVERT(activity_class_5min USING utf8)'].str.split('/'))
signal_activity_class_5min = pd.DataFrame(signal_list).drop(288, axis=1).astype('float')
y_df = data_convert['DIAG_NM'] 
signal_activity_class_5min = pd.concat([signal_activity_class_5min, y_df], axis=1)
signal_activity_class_5min

,0,1,2,3,4,5,6,7,8,9,...,279,280,281,282,283,284,285,286,287,DIAG_NM
0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,...,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
3,2.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
9323,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
9324,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
9325,1.0,1.0,1.0,2.0,3.0,2.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN


#### - CONVERT(activity_met_1min USING utf8)

In [8]:
signal_list = list(data_convert['CONVERT(activity_met_1min USING utf8)'].str.split('/'))
signal_activity_met_1min = pd.DataFrame(signal_list).drop(1440, axis=1).astype('float')
signal_activity_met_1min = pd.concat([signal_activity_met_1min, y_df], axis=1)
signal_activity_met_1min

,0,1,2,3,4,5,6,7,8,9,...,1431,1432,1433,1434,1435,1436,1437,1438,1439,DIAG_NM
0,0.9,0.9,1.4,1.9,1.1,0.9,0.9,1.1,1.3,1.0,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
1,1.2,1.1,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
2,1.1,1.1,1.2,1.1,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
4,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,1.1,1.4,1.2,0.9,1.2,1.1,0.9,0.9,0.9,1.0,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
9323,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
9324,0.9,0.9,0.9,0.9,1.1,0.9,0.9,1.1,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
9325,1.1,0.9,0.9,0.9,1.0,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN


#### - CONVERT(sleep_hr_5min USING utf8)

In [9]:
signal_list = list(data_convert['CONVERT(sleep_hr_5min USING utf8)'].str.split('/'))
signal_sleep_hr_5min = pd.DataFrame(signal_list)
signal_sleep_hr_5min

,0,1,2,3,4,5,6,7,8,9,...,172,173,174,175,176,177,178,179,180,181
0,0,73,73,73,72,71,70,71,71,71,...,None,None,None,None,None,None,None,None,None,None
1,69,70,69,69,70,72,71,72,70,69,...,None,None,None,None,None,None,None,None,None,None
2,0,74,73,73,74,74,74,71,71,70,...,None,None,None,None,None,None,None,None,None,None
3,73,70,71,72,75,75,73,70,70,70,...,None,None,None,None,None,None,None,None,None,None
4,0,0,0,0,0,0,0,0,69,69,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,0,54,54,54,55,56,0,0,55,52,...,None,None,None,None,None,None,None,None,None,None
9323,68,66,67,67,68,69,69,70,71,71,...,None,None,None,None,None,None,None,None,None,None
9324,0,0,65,65,66,67,69,72,73,72,...,None,None,None,None,None,None,None,None,None,None
9325,0,0,0,64,64,64,64,65,65,66,...,None,None,None,None,None,None,None,None,None,None


In [10]:
signal_sleep_hr_5min = signal_sleep_hr_5min.drop(181, axis=1).transpose()
# 결측치 처리 Nan, '', ' ' -> 우선 다 int type 0으로 replace 
signal_sleep_hr_5min = signal_sleep_hr_5min.fillna(0) 
signal_sleep_hr_5min = signal_sleep_hr_5min.replace('', 0)
signal_sleep_hr_5min = signal_sleep_hr_5min.replace(' ', 0)
signal_sleep_hr_5min = signal_sleep_hr_5min.astype('float')

In [11]:
signal_sleep_hr_5min  = signal_sleep_hr_5min.transpose()
signal_sleep_hr_5min  = signal_sleep_hr_5min.drop(180,axis =1) #convert sleep data 3개 길이 맞추기
signal_sleep_hr_5min = pd.concat([signal_sleep_hr_5min, y_df], axis=1)
signal_sleep_hr_5min

,0,1,2,3,4,5,6,7,8,9,...,171,172,173,174,175,176,177,178,179,DIAG_NM
0,0.0,73.0,73.0,73.0,72.0,71.0,70.0,71.0,71.0,71.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
1,69.0,70.0,69.0,69.0,70.0,72.0,71.0,72.0,70.0,69.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
2,0.0,74.0,73.0,73.0,74.0,74.0,74.0,71.0,71.0,70.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
3,73.0,70.0,71.0,72.0,75.0,75.0,73.0,70.0,70.0,70.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,69.0,69.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,0.0,54.0,54.0,54.0,55.0,56.0,0.0,0.0,55.0,52.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9323,68.0,66.0,67.0,67.0,68.0,69.0,69.0,70.0,71.0,71.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9324,0.0,0.0,65.0,65.0,66.0,67.0,69.0,72.0,73.0,72.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9325,0.0,0.0,0.0,64.0,64.0,64.0,64.0,65.0,65.0,66.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN


#### - CONVERT(sleep_hypnogram_5min USING utf8)

In [12]:
signal_list = list(data_convert['CONVERT(sleep_hypnogram_5min USING utf8)'].str.split('/'))
signal_sleep_hypnogram_5min = pd.DataFrame(signal_list)

signal_sleep_hypnogram_5min = signal_sleep_hypnogram_5min.drop(180, axis=1).transpose()
# 결측치 처리 Nan, '', ' ' -> 우선 다 int type 0으로 replace 
signal_sleep_hypnogram_5min = signal_sleep_hypnogram_5min.fillna(0) 
signal_sleep_hypnogram_5min = signal_sleep_hypnogram_5min.replace('', 0)
signal_sleep_hypnogram_5min = signal_sleep_hypnogram_5min.replace(' ', 0)
signal_sleep_hypnogram_5min = signal_sleep_hypnogram_5min.astype('float')

In [13]:
signal_sleep_hypnogram_5min  = signal_sleep_hypnogram_5min.transpose()
signal_sleep_hypnogram_5min = pd.concat([signal_sleep_hypnogram_5min, y_df], axis=1)
signal_sleep_hypnogram_5min

,0,1,2,3,4,5,6,7,8,9,...,171,172,173,174,175,176,177,178,179,DIAG_NM
0,4.0,2.0,4.0,3.0,3.0,1.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
1,2.0,4.0,2.0,2.0,2.0,2.0,3.0,1.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
2,4.0,2.0,4.0,4.0,1.0,1.0,1.0,4.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
3,4.0,4.0,4.0,4.0,3.0,3.0,3.0,2.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
4,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,4.0,4.0,4.0,2.0,2.0,2.0,2.0,4.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9323,4.0,4.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9324,4.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9325,4.0,4.0,4.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN


#### - CONVERT(sleep_rmssd_5min USING utf8)

In [14]:
signal_list = list(data_convert['CONVERT(sleep_rmssd_5min USING utf8)'].str.split('/'))
signal_sleep_rmssd_5min = pd.DataFrame(signal_list)

signal_sleep_rmssd_5min = signal_sleep_rmssd_5min.drop(181, axis=1).transpose()
# 결측치 처리 Nan, '', ' ' -> 우선 다 int type 0으로 replace 
signal_sleep_rmssd_5min = signal_sleep_rmssd_5min.fillna(0) 
signal_sleep_rmssd_5min = signal_sleep_rmssd_5min.replace('', 0)
signal_sleep_rmssd_5min = signal_sleep_rmssd_5min.replace(' ', 0)
signal_sleep_rmssd_5min = signal_sleep_rmssd_5min.astype('float')

In [15]:
signal_sleep_rmssd_5min  = signal_sleep_rmssd_5min.transpose()
signal_sleep_rmssd_5min  = signal_sleep_rmssd_5min.drop(180,axis =1) #convert sleep data 3개 길이 맞추기
signal_sleep_rmssd_5min = pd.concat([signal_sleep_rmssd_5min, y_df], axis=1)
signal_sleep_rmssd_5min

,0,1,2,3,4,5,6,7,8,9,...,171,172,173,174,175,176,177,178,179,DIAG_NM
0,0.0,10.0,10.0,10.0,11.0,11.0,10.0,12.0,18.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
1,23.0,23.0,26.0,24.0,18.0,13.0,15.0,14.0,17.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
2,0.0,11.0,14.0,20.0,13.0,14.0,14.0,16.0,27.0,29.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
3,24.0,28.0,19.0,17.0,12.0,10.0,17.0,20.0,23.0,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,0.0,35.0,39.0,28.0,26.0,41.0,0.0,0.0,52.0,31.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9323,26.0,16.0,19.0,18.0,16.0,18.0,19.0,17.0,15.0,16.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9324,0.0,0.0,13.0,12.0,12.0,13.0,12.0,12.0,12.0,16.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9325,0.0,0.0,0.0,14.0,17.0,18.0,14.0,14.0,17.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN


In [16]:
#signal1 = signal_activity_class_5min.values.tolist()
#signal2 = signal_activity_met_1min.values.tolist()
#signal3 = signal_sleep_hr_5min.values.tolist()
#signal4 = signal_sleep_hypnogram_5min.values.tolist()
#signal5 = signal_sleep_rmssd_5min.values.tolist()

### CONVERT Dadaset 3개를 하나로 합치기


# Data Split

In [17]:
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, GridSearchCV

# y 변수와 X 변수 분할 관리 
input_data_y = y_df.copy()
input_data_X1 = signal_activity_class_5min.drop(labels = 'DIAG_NM', axis = 1)
input_data_X2 = signal_activity_met_1min.drop(labels = 'DIAG_NM', axis = 1)
input_data_X3 = signal_sleep_hr_5min.drop(labels = 'DIAG_NM', axis = 1)
input_data_X4 = signal_sleep_hypnogram_5min.drop(labels = 'DIAG_NM', axis = 1)
input_data_X5 = signal_sleep_rmssd_5min.drop(labels = 'DIAG_NM', axis = 1)

In [18]:
# LabelEncoder
# y data를 LabelEncdoer로 한 번 더 labeling 합니다 -> 100만원, 10000만원... = 0, 1, ... 
# 추후에 DL에서 output을 맞춰주기 위함입니다, 최종 마지막에서 원래 label값(100만원, 1000만원...)으로 되돌립니다
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
input_data_y = le.fit_transform(list(input_data_y))

print("original gain_label \n", le.classes_)
print("gain_lable label \n", set(input_data_y))

original gain_label 
 ['CN' 'Dem' 'MCI']
gain_lable label 
 {0, 1, 2}


- train data, test data를 8:2로 나누기

In [19]:
X1_train, X1_test, y1_train, y1_test = train_test_split(input_data_X1, input_data_y, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=34)
X2_train, X2_test, y2_train, y2_test = train_test_split(input_data_X2, input_data_y, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=34)
X3_train, X3_test, y3_train, y3_test = train_test_split(input_data_X3, input_data_y, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=34)
X4_train, X4_test, y4_train, y4_test = train_test_split(input_data_X4, input_data_y, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=34)
X5_train, X5_test, y5_train, y5_test = train_test_split(input_data_X5, input_data_y, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=34)

- train data를 train data, validation data로 8:2 나누기

In [20]:
X1_train, X1_val, y1_train, y1_val = train_test_split(X1_train, y1_train, test_size=0.2, stratify=y1_train, random_state=34)
X2_train, X2_val, y2_train, y2_val = train_test_split(X2_train, y2_train, test_size=0.2, stratify=y2_train, random_state=34)
X3_train, X3_val, y3_train, y3_val = train_test_split(X3_train, y3_train, test_size=0.2, stratify=y3_train, random_state=34)
X4_train, X4_val, y4_train, y4_val = train_test_split(X4_train, y4_train, test_size=0.2, stratify=y4_train, random_state=34)
X5_train, X5_val, y5_train, y5_val = train_test_split(X5_train, y5_train, test_size=0.2, stratify=y5_train, random_state=34)

In [21]:
print('훈련용 signal_sleep_hr_5min: {}'.format(len(X3_train)))
print('훈련용 signal_sleep_hr_5min: {}'.format(len(X3_val)))
print('테스트용 signal_sleep_hr_5min : {}'.format(len(X3_test)))
num_classes = len(set(y3_train))
print('label: {}'.format(num_classes))

훈련용 signal_sleep_hr_5min: 5968
훈련용 signal_sleep_hr_5min: 1493
테스트용 signal_sleep_hr_5min : 1866
label: 3


In [22]:
unique_elements, counts_elements = np.unique(y3_train, return_counts=True)
print("signal_sleep_hr_5min train에서 각 레이블에 대한 빈도수:")
print(np.asarray((unique_elements, counts_elements)))

unique_elements, counts_elements = np.unique(y3_val, return_counts=True)
print("signal_sleep_hr_5min validation에서 각 레이블에 대한 빈도수:")
print(np.asarray((unique_elements, counts_elements)))

unique_elements, counts_elements = np.unique(y3_test, return_counts=True)
print("signal_sleep_hr_5min test에서 각 레이블에 대한 빈도수:")
print(np.asarray((unique_elements, counts_elements)))

signal_sleep_hr_5min train에서 각 레이블에 대한 빈도수:
[[   0    1    2]
 [3820  410 1738]]
signal_sleep_hr_5min validation에서 각 레이블에 대한 빈도수:
[[  0   1   2]
 [955 103 435]]
signal_sleep_hr_5min test에서 각 레이블에 대한 빈도수:
[[   0    1    2]
 [1194  129  543]]


In [26]:
X1_train

,0,1,2,3,4,5,6,7,8,9,...,278,279,280,281,282,283,284,285,286,287
8878,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0
7363,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0
6855,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7628,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7984,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0
7135,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
897,4.0,4.0,4.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4330,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


# Modeling - RNN

In [24]:
print("X1 :" ,X1_train.shape)
print("y1 :" ,y1_train.shape)
print("X2 :" ,X2_train.shape)
print("y2 :" ,y2_train.shape)
print("X3 :" ,X3_train.shape)
print("y3 :" ,y3_train.shape)
print("X4 :" ,X4_train.shape)
print("y4 :" ,y4_train.shape)
print("X5 :" ,X5_train.shape)
print("y5 :" ,y5_train.shape)

X1 : (5968, 288)
y1 : (5968,)
X2 : (5968, 1440)
y2 : (5968,)
X3 : (5968, 180)
y3 : (5968,)
X4 : (5968, 180)
y4 : (5968,)
X5 : (5968, 180)
y5 : (5968,)


In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Activation, SimpleRNN
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from keras.layers import Bidirectional
from keras import optimizers
from sklearn.metrics import accuracy_score
from keras.layers import Bidirectional
from keras.wrappers.scikit_learn import KerasClassifier

import tensorflow as tf

AttributeError: module 'tensorflow.compat.v2.__internal__' has no attribute 'register_clear_session_function'

### 기본 RNN 모델

In [47]:
def vanilla_rnn():
    model = Sequential()
    model.add(SimpleRNN(20, input_shape = (180,1), return_sequences = False)) #input shape에 열 개수 넣어주면 됨
    model.add(Dense(3)) #target 개수 = 3
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss =  'sparse_categorical_crossentropy', optimizer = adam, metrics = ['accuracy']) #label encoding 했기때문에  'sparse_categorical_crossentropy' 사용
    
    model.summary()
    return model

- CONVERT(sleep_hr_5min USING utf8) data

In [49]:
model = KerasClassifier(build_fn = vanilla_rnn, epochs = 20, batch_size = 64, verbose = 1)
model.fit(X3_train, y3_train)
pred_train = model.predict(X3_train)
pred_val = model.predict(X3_val)
print("train set의 accuracy : {}, validation set의 예측결과 : {}" .format(accuracy_score(y3_train, pred_train), accuracy_score(y3_val, pred_val)))

pred_test = model.predict(X3_test)
print("최종 평가(test set): {}".format(accuracy_score(y3_test, pred_test)))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 20)                440       
                                                                 
 dense_3 (Dense)             (None, 3)                 63        
                                                                 
 activation_3 (Activation)   (None, 3)                 0         
                                                                 
Total params: 503
Trainable params: 503
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20


C:\Users\yeada\AppData\Local\Temp/ipykernel_12704/783657071.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = vanilla_rnn, epochs = 20, batch_size = 64, verbose = 1)
C:\Users\yeada\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


94/94 [==============================] - 2s 15ms/step - loss: 0.9274 - accuracy: 0.5680
Epoch 2/20
94/94 [==============================] - 1s 14ms/step - loss: 0.8276 - accuracy: 0.6389
Epoch 3/20
94/94 [==============================] - 1s 14ms/step - loss: 0.8259 - accuracy: 0.6377
Epoch 4/20
94/94 [==============================] - 1s 15ms/step - loss: 0.8245 - accuracy: 0.6391
Epoch 5/20
94/94 [==============================] - 1s 15ms/step - loss: 0.8342 - accuracy: 0.6335
Epoch 6/20
94/94 [==============================] - 1s 14ms/step - loss: 0.8246 - accuracy: 0.6399
Epoch 7/20
94/94 [==============================] - 1s 14ms/step - loss: 0.8230 - accuracy: 0.6401
Epoch 8/20
94/94 [==============================] - 1s 15ms/step - loss: 0.8201 - accuracy: 0.6416
Epoch 9/20
94/94 [==============================] - 1s 15ms/step - loss: 0.8197 - accuracy: 0.6408
Epoch 10/20
94/94 [==============================] - 2s 16ms/step - loss: 0.8184 - accuracy: 0.6416
Epoch 11/20
94/94 [=

- CONVERT(sleep_hypnogram_5min USING utf8) data

In [50]:
model = KerasClassifier(build_fn = vanilla_rnn, epochs = 20, batch_size = 64, verbose = 1)
model.fit(X4_train, y4_train)
pred_train = model.predict(X4_train)
pred_val = model.predict(X4_val)
print("train set의 accuracy : {}, validation set의 예측결과 : {}" .format(accuracy_score(y4_train, pred_train), accuracy_score(y4_val, pred_val)))

pred_test = model.predict(X4_test)
print("최종 평가(test set): {}".format(accuracy_score(y4_test, pred_test)))

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (None, 20)                440       
                                                                 
 dense_4 (Dense)             (None, 3)                 63        
                                                                 
 activation_4 (Activation)   (None, 3)                 0         
                                                                 
Total params: 503
Trainable params: 503
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20


C:\Users\yeada\AppData\Local\Temp/ipykernel_12704/10281997.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = vanilla_rnn, epochs = 20, batch_size = 64, verbose = 1)
C:\Users\yeada\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


94/94 [==============================] - 2s 15ms/step - loss: 0.8992 - accuracy: 0.6149
Epoch 2/20
94/94 [==============================] - 1s 16ms/step - loss: 0.8278 - accuracy: 0.6394
Epoch 3/20
94/94 [==============================] - 1s 16ms/step - loss: 0.8250 - accuracy: 0.6377
Epoch 4/20
94/94 [==============================] - 1s 14ms/step - loss: 0.8238 - accuracy: 0.6397
Epoch 5/20
94/94 [==============================] - 1s 15ms/step - loss: 0.8230 - accuracy: 0.6399
Epoch 6/20
94/94 [==============================] - 1s 16ms/step - loss: 0.8211 - accuracy: 0.6413
Epoch 7/20
94/94 [==============================] - 2s 16ms/step - loss: 0.8215 - accuracy: 0.6399
Epoch 8/20
94/94 [==============================] - 1s 15ms/step - loss: 0.8203 - accuracy: 0.6381
Epoch 9/20
94/94 [==============================] - 1s 16ms/step - loss: 0.8176 - accuracy: 0.6387
Epoch 10/20
94/94 [==============================] - 1s 15ms/step - loss: 0.8146 - accuracy: 0.6402
Epoch 11/20
94/94 [=

- CONVERT(sleep_rmssd_5min USING utf8) data

In [51]:
model = KerasClassifier(build_fn = vanilla_rnn, epochs = 20, batch_size = 64, verbose = 1)
model.fit(X5_train, y5_train)
pred_train = model.predict(X5_train)
pred_val = model.predict(X5_val)
print("train set의 accuracy : {}, validation set의 예측결과 : {}" .format(accuracy_score(y5_train, pred_train), accuracy_score(y5_val, pred_val)))

pred_test = model.predict(X5_test)
print("최종 평가(test set): {}".format(accuracy_score(y5_test, pred_test)))

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_3 (SimpleRNN)    (None, 20)                440       
                                                                 
 dense_5 (Dense)             (None, 3)                 63        
                                                                 
 activation_5 (Activation)   (None, 3)                 0         
                                                                 
Total params: 503
Trainable params: 503
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20


C:\Users\yeada\AppData\Local\Temp/ipykernel_12704/2146687755.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = vanilla_rnn, epochs = 20, batch_size = 64, verbose = 1)
C:\Users\yeada\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


94/94 [==============================] - 2s 15ms/step - loss: 0.8694 - accuracy: 0.6222
Epoch 2/20
94/94 [==============================] - 1s 15ms/step - loss: 0.8338 - accuracy: 0.6366
Epoch 3/20
94/94 [==============================] - 1s 15ms/step - loss: 0.8269 - accuracy: 0.6401
Epoch 4/20
94/94 [==============================] - 1s 15ms/step - loss: 0.8257 - accuracy: 0.6401
Epoch 5/20
94/94 [==============================] - 1s 15ms/step - loss: 0.8283 - accuracy: 0.6386
Epoch 6/20
94/94 [==============================] - 1s 16ms/step - loss: 0.8256 - accuracy: 0.6409
Epoch 7/20
94/94 [==============================] - 1s 15ms/step - loss: 0.8249 - accuracy: 0.6408
Epoch 8/20
94/94 [==============================] - 1s 15ms/step - loss: 0.8256 - accuracy: 0.6404
Epoch 9/20
94/94 [==============================] - 2s 16ms/step - loss: 0.8249 - accuracy: 0.6409
Epoch 10/20
94/94 [==============================] - 1s 14ms/step - loss: 0.8241 - accuracy: 0.6408
Epoch 11/20
94/94 [=

### 다층 RNN 모델

In [52]:
def stacked_vanilla_rnn():
    model = Sequential()
    model.add(SimpleRNN(50, input_shape = (180,1), return_sequences = True))   # return_sequences parameter has to be set True to stack
    model.add(SimpleRNN(50, return_sequences = False))
    model.add(Dense(3))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    model.summary()
    return model

- CONVERT(sleep_hr_5min USING utf8) data

In [53]:
model = KerasClassifier(build_fn = stacked_vanilla_rnn, epochs = 10, batch_size = 64, verbose = 1)
model.fit(X3_train, y3_train)
pred_train = model.predict(X3_train)
pred_val = model.predict(X3_val)
print("train set의 accuracy : {}, validation set의 예측결과 : {}" .format(accuracy_score(y3_train, pred_train), accuracy_score(y3_val, pred_val)))

pred_test = model.predict(X3_test)
print("최종 평가(test set): {}".format(accuracy_score(y3_test, pred_test)))

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_4 (SimpleRNN)    (None, 180, 50)           2600      
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 50)                5050      
                                                                 
 dense_6 (Dense)             (None, 3)                 153       
                                                                 
 activation_6 (Activation)   (None, 3)                 0         
                                                                 
Total params: 7,803
Trainable params: 7,803
Non-trainable params: 0
_________________________________________________________________


C:\Users\yeada\AppData\Local\Temp/ipykernel_12704/3686456303.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = stacked_vanilla_rnn, epochs = 10, batch_size = 64, verbose = 1)
C:\Users\yeada\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/10
94/94 [==============================] - 5s 35ms/step - loss: 0.8450 - accuracy: 0.6278
Epoch 2/10
94/94 [==============================] - 3s 35ms/step - loss: 0.8358 - accuracy: 0.6342
Epoch 3/10
94/94 [==============================] - 3s 35ms/step - loss: 0.8245 - accuracy: 0.6402
Epoch 4/10
94/94 [==============================] - 4s 37ms/step - loss: 0.8221 - accuracy: 0.6408
Epoch 5/10
94/94 [==============================] - 3s 36ms/step - loss: 0.8207 - accuracy: 0.6406
Epoch 6/10
94/94 [==============================] - 3s 36ms/step - loss: 0.8200 - accuracy: 0.6411
Epoch 7/10
94/94 [==============================] - 4s 38ms/step - loss: 0.8177 - accuracy: 0.6426
Epoch 8/10
94/94 [==============================] - 4s 38ms/step - loss: 0.8164 - accuracy: 0.6394
Epoch 9/10
94/94 [==============================] - 4s 38ms/step - loss: 0.8160 - accuracy: 0.6409
Epoch 10/10
47/47 [==============================] - 1s 10ms/step
train set의 accuracy : 0.6425938337801609, v

- CONVERT(sleep_hypnogram_5min USING utf8) data

In [54]:
model = KerasClassifier(build_fn = stacked_vanilla_rnn, epochs = 10, batch_size = 64, verbose = 1)
model.fit(X4_train, y4_train)
pred_train = model.predict(X4_train)
pred_val = model.predict(X4_val)
print("train set의 accuracy : {}, validation set의 예측결과 : {}" .format(accuracy_score(y4_train, pred_train), accuracy_score(y4_val, pred_val)))

pred_test = model.predict(X4_test)
print("최종 평가(test set): {}".format(accuracy_score(y4_test, pred_test)))

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_6 (SimpleRNN)    (None, 180, 50)           2600      
                                                                 
 simple_rnn_7 (SimpleRNN)    (None, 50)                5050      
                                                                 
 dense_7 (Dense)             (None, 3)                 153       
                                                                 
 activation_7 (Activation)   (None, 3)                 0         
                                                                 
Total params: 7,803
Trainable params: 7,803
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


C:\Users\yeada\AppData\Local\Temp/ipykernel_12704/4067150052.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = stacked_vanilla_rnn, epochs = 10, batch_size = 64, verbose = 1)
C:\Users\yeada\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


94/94 [==============================] - 5s 36ms/step - loss: 0.8341 - accuracy: 0.6324
Epoch 2/10
94/94 [==============================] - 3s 36ms/step - loss: 0.8259 - accuracy: 0.6402
Epoch 3/10
94/94 [==============================] - 3s 36ms/step - loss: 0.8333 - accuracy: 0.6372
Epoch 4/10
94/94 [==============================] - 3s 37ms/step - loss: 0.8322 - accuracy: 0.6401
Epoch 5/10
94/94 [==============================] - 3s 36ms/step - loss: 0.8267 - accuracy: 0.6394
Epoch 6/10
94/94 [==============================] - 3s 36ms/step - loss: 0.8277 - accuracy: 0.6396
Epoch 7/10
94/94 [==============================] - 3s 36ms/step - loss: 0.8245 - accuracy: 0.6402
Epoch 8/10
94/94 [==============================] - 3s 37ms/step - loss: 0.8239 - accuracy: 0.6409
Epoch 9/10
94/94 [==============================] - 3s 36ms/step - loss: 0.8254 - accuracy: 0.6419
Epoch 10/10
47/47 [==============================] - 0s 10ms/step
train set의 accuracy : 0.6419235924932976, validation s

- CONVERT(sleep_rmssd_5min USING utf8) data

In [55]:
model = KerasClassifier(build_fn = stacked_vanilla_rnn, epochs = 10, batch_size = 64, verbose = 1)
model.fit(X5_train, y5_train)
pred_train = model.predict(X5_train)
pred_val = model.predict(X5_val)
print("train set의 accuracy : {}, validation set의 예측결과 : {}" .format(accuracy_score(y5_train, pred_train), accuracy_score(y5_val, pred_val)))

pred_test = model.predict(X5_test)
print("최종 평가(test set): {}".format(accuracy_score(y5_test, pred_test)))

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_8 (SimpleRNN)    (None, 180, 50)           2600      
                                                                 
 simple_rnn_9 (SimpleRNN)    (None, 50)                5050      
                                                                 
 dense_8 (Dense)             (None, 3)                 153       
                                                                 
 activation_8 (Activation)   (None, 3)                 0         
                                                                 
Total params: 7,803
Trainable params: 7,803
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


C:\Users\yeada\AppData\Local\Temp/ipykernel_12704/3495850958.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = stacked_vanilla_rnn, epochs = 10, batch_size = 64, verbose = 1)
C:\Users\yeada\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


94/94 [==============================] - 7s 52ms/step - loss: 0.8503 - accuracy: 0.6156
Epoch 2/10
94/94 [==============================] - 5s 52ms/step - loss: 0.8257 - accuracy: 0.6406
Epoch 3/10
94/94 [==============================] - 5s 52ms/step - loss: 0.8238 - accuracy: 0.6411
Epoch 4/10
94/94 [==============================] - 5s 51ms/step - loss: 0.8233 - accuracy: 0.6416
Epoch 5/10
94/94 [==============================] - 5s 51ms/step - loss: 0.8231 - accuracy: 0.6419
Epoch 6/10
94/94 [==============================] - 5s 52ms/step - loss: 0.8205 - accuracy: 0.6419
Epoch 7/10
94/94 [==============================] - 5s 52ms/step - loss: 0.8849 - accuracy: 0.6099
Epoch 8/10
94/94 [==============================] - 5s 52ms/step - loss: 0.8303 - accuracy: 0.6397
Epoch 9/10
94/94 [==============================] - 5s 52ms/step - loss: 0.8526 - accuracy: 0.6292
Epoch 10/10
47/47 [==============================] - 1s 20ms/step
train set의 accuracy : 0.6407506702412868, validation s

### 단층 LSTM 모델

In [27]:
def lstm():
    model = Sequential()
    model.add(LSTM(200, input_shape = (180,1), return_sequences = False))
    model.add(Dense(3))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    model.summary()
    return model

- CONVERT(sleep_hr_5min USING utf8) data

In [28]:
model = KerasClassifier(build_fn = lstm, epochs = 10, batch_size = 64, verbose = 1)
model.fit(X3_train, y3_train)
pred_train = model.predict(X3_train)
pred_val = model.predict(X3_val)
print("train set의 accuracy : {}, validation set의 예측결과 : {}" .format(accuracy_score(y3_train, pred_train), accuracy_score(y3_val, pred_val)))

pred_test = model.predict(X3_test)
print("최종 평가(test set): {}".format(accuracy_score(y3_test, pred_test)))

NameError: name 'KerasClassifier' is not defined

- CONVERT(sleep_hypnogram_5min USING utf8) data 

In [58]:
model = KerasClassifier(build_fn = lstm, epochs = 10, batch_size = 64, verbose = 1)
model.fit(X4_train, y4_train)
pred_train = model.predict(X4_train)
pred_val = model.predict(X4_val)
print("train set의 accuracy : {}, validation set의 예측결과 : {}" .format(accuracy_score(y4_train, pred_train), accuracy_score(y4_val, pred_val)))

pred_test = model.predict(X4_test)
print("최종 평가(test set): {}".format(accuracy_score(y4_test, pred_test)))

C:\Users\yeada\AppData\Local\Temp/ipykernel_12704/888943653.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = lstm, epochs = 10, batch_size = 64, verbose = 1)


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 200)               161600    
                                                                 
 dense_10 (Dense)            (None, 3)                 603       
                                                                 
 activation_10 (Activation)  (None, 3)                 0         
                                                                 
Total params: 162,203
Trainable params: 162,203
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


C:\Users\yeada\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


94/94 [==============================] - 40s 397ms/step - loss: 0.8490 - accuracy: 0.6352
Epoch 2/10
94/94 [==============================] - 39s 411ms/step - loss: 0.8183 - accuracy: 0.6371
Epoch 3/10
94/94 [==============================] - 39s 413ms/step - loss: 0.8121 - accuracy: 0.6391
Epoch 4/10
94/94 [==============================] - 39s 415ms/step - loss: 0.8104 - accuracy: 0.6386
Epoch 5/10
94/94 [==============================] - 39s 413ms/step - loss: 0.8106 - accuracy: 0.6401
Epoch 6/10
94/94 [==============================] - 39s 413ms/step - loss: 0.8079 - accuracy: 0.6402
Epoch 7/10
94/94 [==============================] - 39s 412ms/step - loss: 0.8120 - accuracy: 0.6396
Epoch 8/10
94/94 [==============================] - 39s 412ms/step - loss: 0.8100 - accuracy: 0.6397
Epoch 9/10
94/94 [==============================] - 39s 411ms/step - loss: 0.8102 - accuracy: 0.6391
Epoch 10/10
47/47 [==============================] - 3s 60ms/step
train set의 accuracy : 0.639410187667

- CONVERT(sleep_rmssd_5min USING utf8) data

In [59]:
model = KerasClassifier(build_fn = lstm, epochs = 10, batch_size = 64, verbose = 1)
model.fit(X5_train, y5_train)
pred_train = model.predict(X5_train)
pred_val = model.predict(X5_val)
print("train set의 accuracy : {}, validation set의 예측결과 : {}" .format(accuracy_score(y5_train, pred_train), accuracy_score(y5_val, pred_val)))

pred_test = model.predict(X5_test)
print("최종 평가(test set): {}".format(accuracy_score(y5_test, pred_test)))

C:\Users\yeada\AppData\Local\Temp/ipykernel_12704/423928694.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = lstm, epochs = 10, batch_size = 64, verbose = 1)
C:\Users\yeada\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 200)               161600    
                                                                 
 dense_11 (Dense)            (None, 3)                 603       
                                                                 
 activation_11 (Activation)  (None, 3)                 0         
                                                                 
Total params: 162,203
Trainable params: 162,203
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
94/94 [==============================] - 29s 294ms/step - loss: 0.8502 - accuracy: 0.6374
Epoch 2/10
94/94 [==============================] - 28s 293ms/step - loss: 0.8224 - accuracy: 0.6399
Epoch 3/10
94/94 [==============================] - 29s 309ms/step - loss: 0.8133 - accuracy: 0.6394
Epoch 4/

### 다층 LSTM 모델

In [60]:
def stacked_lstm():
    model = Sequential()
    model.add(LSTM(20, input_shape = (180,1), return_sequences = True))
    model.add(LSTM(20, return_sequences = False))
    model.add(Dense(3))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    model.summary()
    return model

- CONVERT(sleep_hr_5min USING utf8) data  

In [61]:
model = KerasClassifier(build_fn = stacked_lstm, epochs = 10, batch_size = 64, verbose = 1)
model.fit(X3_train, y3_train)
pred_train = model.predict(X3_train)
pred_val = model.predict(X3_val)
print("train set의 accuracy : {}, validation set의 예측결과 : {}" .format(accuracy_score(y3_train, pred_train), accuracy_score(y3_val, pred_val)))

pred_test = model.predict(X3_test)
print("최종 평가(test set): {}".format(accuracy_score(y3_test, pred_test)))

C:\Users\yeada\AppData\Local\Temp/ipykernel_12704/3751255507.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = stacked_lstm, epochs = 10, batch_size = 64, verbose = 1)


Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 180, 20)           1760      
                                                                 
 lstm_6 (LSTM)               (None, 20)                3280      
                                                                 
 dense_12 (Dense)            (None, 3)                 63        
                                                                 
 activation_12 (Activation)  (None, 3)                 0         
                                                                 
Total params: 5,103
Trainable params: 5,103
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


C:\Users\yeada\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


94/94 [==============================] - 13s 91ms/step - loss: 0.8762 - accuracy: 0.6369
Epoch 2/10
94/94 [==============================] - 9s 91ms/step - loss: 0.8140 - accuracy: 0.6386
Epoch 3/10
94/94 [==============================] - 8s 90ms/step - loss: 0.8141 - accuracy: 0.6391
Epoch 4/10
94/94 [==============================] - 9s 92ms/step - loss: 0.8119 - accuracy: 0.6401
Epoch 5/10
94/94 [==============================] - 9s 92ms/step - loss: 0.8089 - accuracy: 0.6397
Epoch 6/10
94/94 [==============================] - 9s 92ms/step - loss: 0.8095 - accuracy: 0.6394
Epoch 7/10
94/94 [==============================] - 9s 92ms/step - loss: 0.8107 - accuracy: 0.6394
Epoch 8/10
94/94 [==============================] - 9s 92ms/step - loss: 0.8103 - accuracy: 0.6402
Epoch 9/10
94/94 [==============================] - 9s 100ms/step - loss: 0.8086 - accuracy: 0.6404
Epoch 10/10
47/47 [==============================] - 1s 24ms/step
train set의 accuracy : 0.640583109919571, validation 

- CONVERT(sleep_hypnogram_5min USING utf8) data

In [66]:
model = KerasClassifier(build_fn = stacked_lstm, epochs = 10, batch_size = 64, verbose = 1)
model.fit(X4_train, y4_train)
pred_train = model.predict(X4_train)
pred_val = model.predict(X4_val)
print("train set의 accuracy : {}, validation set의 예측결과 : {}" .format(accuracy_score(y4_train, pred_train), accuracy_score(y4_val, pred_val)))

pred_test = model.predict(X4_test)
print("최종 평가(test set): {}".format(accuracy_score(y4_test, pred_test)))

C:\Users\yeada\AppData\Local\Temp/ipykernel_12704/3795191726.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = stacked_lstm, epochs = 10, batch_size = 64, verbose = 1)


Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_11 (LSTM)              (None, 180, 20)           1760      
                                                                 
 lstm_12 (LSTM)              (None, 20)                3280      
                                                                 
 dense_15 (Dense)            (None, 3)                 63        
                                                                 
 activation_15 (Activation)  (None, 3)                 0         
                                                                 
Total params: 5,103
Trainable params: 5,103
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


C:\Users\yeada\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


94/94 [==============================] - 12s 92ms/step - loss: 0.8779 - accuracy: 0.6374
Epoch 2/10
94/94 [==============================] - 9s 93ms/step - loss: 0.8149 - accuracy: 0.6386
Epoch 3/10
94/94 [==============================] - 9s 93ms/step - loss: 0.8106 - accuracy: 0.6384
Epoch 4/10
94/94 [==============================] - 9s 93ms/step - loss: 0.8077 - accuracy: 0.6397
Epoch 5/10
94/94 [==============================] - 9s 94ms/step - loss: 0.8083 - accuracy: 0.6401
Epoch 6/10
94/94 [==============================] - 9s 94ms/step - loss: 0.8084 - accuracy: 0.6401
Epoch 7/10
94/94 [==============================] - 9s 92ms/step - loss: 0.8072 - accuracy: 0.6401
Epoch 8/10
94/94 [==============================] - 9s 92ms/step - loss: 0.8062 - accuracy: 0.6401
Epoch 9/10
94/94 [==============================] - 9s 93ms/step - loss: 0.8063 - accuracy: 0.6401
Epoch 10/10
47/47 [==============================] - 1s 21ms/step
train set의 accuracy : 0.6400804289544236, validation 

- CONVERT(sleep_rmssd_5min USING utf8) data

In [65]:
model = KerasClassifier(build_fn = stacked_lstm, epochs = 10, batch_size = 64, verbose = 1)
model.fit(X5_train, y5_train)
pred_train = model.predict(X5_train)
pred_val = model.predict(X5_val)
print("train set의 accuracy : {}, validation set의 예측결과 : {}" .format(accuracy_score(y5_train, pred_train), accuracy_score(y5_val, pred_val)))

pred_test = model.predict(X5_test)
print("최종 평가(test set): {}".format(accuracy_score(y5_test, pred_test)))

C:\Users\yeada\AppData\Local\Temp/ipykernel_12704/2983830214.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = stacked_lstm, epochs = 10, batch_size = 64, verbose = 1)


Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 180, 20)           1760      
                                                                 
 lstm_10 (LSTM)              (None, 20)                3280      
                                                                 
 dense_14 (Dense)            (None, 3)                 63        
                                                                 
 activation_14 (Activation)  (None, 3)                 0         
                                                                 
Total params: 5,103
Trainable params: 5,103
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


C:\Users\yeada\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


94/94 [==============================] - 10s 69ms/step - loss: 0.8791 - accuracy: 0.6361
Epoch 2/10
94/94 [==============================] - 6s 68ms/step - loss: 0.8164 - accuracy: 0.6367
Epoch 3/10
94/94 [==============================] - 7s 69ms/step - loss: 0.8106 - accuracy: 0.6394
Epoch 4/10
94/94 [==============================] - 6s 69ms/step - loss: 0.8081 - accuracy: 0.6404
Epoch 5/10
94/94 [==============================] - 6s 69ms/step - loss: 0.8074 - accuracy: 0.6409
Epoch 6/10
94/94 [==============================] - 7s 69ms/step - loss: 0.8110 - accuracy: 0.6402
Epoch 7/10
94/94 [==============================] - 6s 69ms/step - loss: 0.8087 - accuracy: 0.6404
Epoch 8/10
94/94 [==============================] - 7s 69ms/step - loss: 0.8076 - accuracy: 0.6408
Epoch 9/10
94/94 [==============================] - 7s 70ms/step - loss: 0.8063 - accuracy: 0.6411
Epoch 10/10
47/47 [==============================] - 1s 18ms/step
train set의 accuracy : 0.6420911528150134, validation 

### 양방향 단층 LSTM 모델

In [67]:
def bidirectional_lstm():
    model = Sequential()
    model.add(Bidirectional(LSTM(20, return_sequences = False), input_shape = (180,1))) #input shape에 열 개수 넣어주면 됨
    model.add(Dense(3)) #target 개수 = 3
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    model.summary()
    return model

- CONVERT(sleep_hr_5min USING utf8) data

In [68]:
model = KerasClassifier(build_fn = bidirectional_lstm, epochs = 10, batch_size = 64, verbose = 1)
model.fit(X3_train, y3_train)
pred_train = model.predict(X3_train)
pred_val = model.predict(X3_val)
print("train set의 accuracy : {}, validation set의 예측결과 : {}" .format(accuracy_score(y3_train, pred_train), accuracy_score(y3_val, pred_val)))

pred_test = model.predict(X3_test)
print("최종 평가(test set): {}".format(accuracy_score(y3_test, pred_test)))

C:\Users\yeada\AppData\Local\Temp/ipykernel_12704/3410170759.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = bidirectional_lstm, epochs = 10, batch_size = 64, verbose = 1)


Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirectio  (None, 40)               3520      
 nal)                                                            
                                                                 
 dense_16 (Dense)            (None, 3)                 123       
                                                                 
 activation_16 (Activation)  (None, 3)                 0         
                                                                 
Total params: 3,643
Trainable params: 3,643
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


C:\Users\yeada\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


94/94 [==============================] - 8s 43ms/step - loss: 0.9071 - accuracy: 0.5756
Epoch 2/10
94/94 [==============================] - 4s 42ms/step - loss: 0.8177 - accuracy: 0.6399
Epoch 3/10
94/94 [==============================] - 4s 43ms/step - loss: 0.8083 - accuracy: 0.6402
Epoch 4/10
94/94 [==============================] - 4s 43ms/step - loss: 0.8038 - accuracy: 0.6419
Epoch 5/10
94/94 [==============================] - 5s 48ms/step - loss: 0.7997 - accuracy: 0.6428
Epoch 6/10
94/94 [==============================] - 5s 51ms/step - loss: 0.7977 - accuracy: 0.6421
Epoch 7/10
94/94 [==============================] - 4s 44ms/step - loss: 0.7966 - accuracy: 0.6428
Epoch 8/10
94/94 [==============================] - 4s 44ms/step - loss: 0.7955 - accuracy: 0.6434
Epoch 9/10
94/94 [==============================] - 5s 52ms/step - loss: 0.7944 - accuracy: 0.6419
Epoch 10/10
47/47 [==============================] - 1s 13ms/step
train set의 accuracy : 0.6435991957104558, validation s

- CONVERT(sleep_hypnogram_5min USING utf8) data

In [69]:
model = KerasClassifier(build_fn = bidirectional_lstm, epochs = 10, batch_size = 64, verbose = 1)
model.fit(X4_train, y4_train)
pred_train = model.predict(X4_train)
pred_val = model.predict(X4_val)
print("train set의 accuracy : {}, validation set의 예측결과 : {}" .format(accuracy_score(y4_train, pred_train), accuracy_score(y4_val, pred_val)))

pred_test = model.predict(X4_test)
print("최종 평가(test set): {}".format(accuracy_score(y4_test, pred_test)))

C:\Users\yeada\AppData\Local\Temp/ipykernel_12704/1152217995.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = bidirectional_lstm, epochs = 10, batch_size = 64, verbose = 1)


Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_3 (Bidirectio  (None, 40)               3520      
 nal)                                                            
                                                                 
 dense_17 (Dense)            (None, 3)                 123       
                                                                 
 activation_17 (Activation)  (None, 3)                 0         
                                                                 
Total params: 3,643
Trainable params: 3,643
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


C:\Users\yeada\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


94/94 [==============================] - 8s 45ms/step - loss: 0.8579 - accuracy: 0.6352
Epoch 2/10
94/94 [==============================] - 4s 39ms/step - loss: 0.8096 - accuracy: 0.6391
Epoch 3/10
94/94 [==============================] - 4s 39ms/step - loss: 0.8073 - accuracy: 0.6392
Epoch 4/10
94/94 [==============================] - 4s 42ms/step - loss: 0.8067 - accuracy: 0.6399
Epoch 5/10
94/94 [==============================] - 5s 49ms/step - loss: 0.8060 - accuracy: 0.6399
Epoch 6/10
94/94 [==============================] - 4s 40ms/step - loss: 0.8061 - accuracy: 0.6397
Epoch 7/10
94/94 [==============================] - 4s 47ms/step - loss: 0.8049 - accuracy: 0.6401
Epoch 8/10
94/94 [==============================] - 4s 42ms/step - loss: 0.8044 - accuracy: 0.6396
Epoch 9/10
94/94 [==============================] - 4s 41ms/step - loss: 0.8019 - accuracy: 0.6406
Epoch 10/10
47/47 [==============================] - 1s 11ms/step
train set의 accuracy : 0.6407506702412868, validation s

- CONVERT(sleep_rmssd_5min USING utf8) data 

In [70]:
model = KerasClassifier(build_fn = bidirectional_lstm, epochs = 10, batch_size = 64, verbose = 1)
model.fit(X5_train, y5_train)
pred_train = model.predict(X5_train)
pred_val = model.predict(X5_val)
print("train set의 accuracy : {}, validation set의 예측결과 : {}" .format(accuracy_score(y5_train, pred_train), accuracy_score(y5_val, pred_val)))

pred_test = model.predict(X5_test)
print("최종 평가(test set): {}".format(accuracy_score(y5_test, pred_test)))

C:\Users\yeada\AppData\Local\Temp/ipykernel_12704/2488008318.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = bidirectional_lstm, epochs = 10, batch_size = 64, verbose = 1)


Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_4 (Bidirectio  (None, 40)               3520      
 nal)                                                            
                                                                 
 dense_18 (Dense)            (None, 3)                 123       
                                                                 
 activation_18 (Activation)  (None, 3)                 0         
                                                                 
Total params: 3,643
Trainable params: 3,643
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


C:\Users\yeada\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


94/94 [==============================] - 7s 44ms/step - loss: 0.8828 - accuracy: 0.5690
Epoch 2/10
94/94 [==============================] - 4s 44ms/step - loss: 0.8159 - accuracy: 0.6369
Epoch 3/10
94/94 [==============================] - 4s 44ms/step - loss: 0.8112 - accuracy: 0.6392
Epoch 4/10
94/94 [==============================] - 4s 44ms/step - loss: 0.8077 - accuracy: 0.6401
Epoch 5/10
94/94 [==============================] - 4s 45ms/step - loss: 0.8061 - accuracy: 0.6408
Epoch 6/10
94/94 [==============================] - 4s 46ms/step - loss: 0.8051 - accuracy: 0.6413
Epoch 7/10
94/94 [==============================] - 4s 46ms/step - loss: 0.8029 - accuracy: 0.6414
Epoch 8/10
94/94 [==============================] - 4s 44ms/step - loss: 0.8019 - accuracy: 0.6416
Epoch 9/10
94/94 [==============================] - 4s 45ms/step - loss: 0.8012 - accuracy: 0.6423
Epoch 10/10
47/47 [==============================] - 1s 12ms/step
train set의 accuracy : 0.6419235924932976, validation s

### 양뱡향 다층 LSTM 모델

In [71]:
# 다층 LSTM을 구현하기 위한 함수
def stacked_bidirectional_lstm():
    model = Sequential()
    model.add(Bidirectional(LSTM(20, return_sequences = True), input_shape = (180,1))) #input shape에 열 개수 넣어주면 됨
    model.add(Bidirectional(LSTM(20, return_sequences = True)))
    model.add(Bidirectional(LSTM(20, return_sequences = False)))
    model.add(Dense(3)) #target 개수 = 3
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    model.summary()
    return model

- CONVERT(sleep_hr_5min USING utf8) data

In [72]:
model = KerasClassifier(build_fn = stacked_bidirectional_lstm, epochs = 10, batch_size = 64, verbose = 1)
model.fit(X3_train, y3_train)
pred_train = model.predict(X3_train)
pred_val = model.predict(X3_val)
print("train set의 accuracy : {}, validation set의 예측결과 : {}" .format(accuracy_score(y3_train, pred_train), accuracy_score(y3_val, pred_val)))

pred_test = model.predict(X3_test)
print("최종 평가(test set): {}".format(accuracy_score(y3_test, pred_test)))

C:\Users\yeada\AppData\Local\Temp/ipykernel_12704/4061416182.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = stacked_bidirectional_lstm, epochs = 10, batch_size = 64, verbose = 1)


Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_5 (Bidirectio  (None, 180, 40)          3520      
 nal)                                                            
                                                                 
 bidirectional_6 (Bidirectio  (None, 180, 40)          9760      
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 40)               9760      
 nal)                                                            
                                                                 
 dense_19 (Dense)            (None, 3)                 123       
                                                                 
 activation_19 (Activation)  (None, 3)                 0         
                                                     

C:\Users\yeada\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


94/94 [==============================] - 25s 185ms/step - loss: 0.8431 - accuracy: 0.6396
Epoch 2/10
94/94 [==============================] - 18s 187ms/step - loss: 0.8079 - accuracy: 0.6397
Epoch 3/10
94/94 [==============================] - 18s 189ms/step - loss: 0.8070 - accuracy: 0.6401
Epoch 4/10
94/94 [==============================] - 18s 191ms/step - loss: 0.8049 - accuracy: 0.6394
Epoch 5/10
94/94 [==============================] - 18s 188ms/step - loss: 0.8043 - accuracy: 0.6396
Epoch 6/10
94/94 [==============================] - 18s 190ms/step - loss: 0.8004 - accuracy: 0.6406
Epoch 7/10
94/94 [==============================] - 18s 189ms/step - loss: 0.8018 - accuracy: 0.6399
Epoch 8/10
94/94 [==============================] - 18s 192ms/step - loss: 0.7981 - accuracy: 0.6408
Epoch 9/10
94/94 [==============================] - 20s 208ms/step - loss: 0.7978 - accuracy: 0.6404
Epoch 10/10
47/47 [==============================] - 3s 57ms/step
train set의 accuracy : 0.641420911528

- CONVERT(sleep_hypnogram_5min USING utf8) data

In [73]:
model = KerasClassifier(build_fn = stacked_bidirectional_lstm, epochs = 10, batch_size = 64, verbose = 1)
model.fit(X4_train, y4_train)
pred_train = model.predict(X4_train)
pred_val = model.predict(X4_val)
print("train set의 accuracy : {}, validation set의 예측결과 : {}" .format(accuracy_score(y4_train, pred_train), accuracy_score(y4_val, pred_val)))

pred_test = model.predict(X4_test)
print("최종 평가(test set): {}".format(accuracy_score(y4_test, pred_test)))

C:\Users\yeada\AppData\Local\Temp/ipykernel_12704/881672587.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = stacked_bidirectional_lstm, epochs = 10, batch_size = 64, verbose = 1)


Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_8 (Bidirectio  (None, 180, 40)          3520      
 nal)                                                            
                                                                 
 bidirectional_9 (Bidirectio  (None, 180, 40)          9760      
 nal)                                                            
                                                                 
 bidirectional_10 (Bidirecti  (None, 40)               9760      
 onal)                                                           
                                                                 
 dense_20 (Dense)            (None, 3)                 123       
                                                                 
 activation_20 (Activation)  (None, 3)                 0         
                                                     

C:\Users\yeada\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


94/94 [==============================] - 24s 143ms/step - loss: 0.8465 - accuracy: 0.6285
Epoch 2/10
94/94 [==============================] - 13s 138ms/step - loss: 0.8091 - accuracy: 0.6391
Epoch 3/10
94/94 [==============================] - 13s 143ms/step - loss: 0.8094 - accuracy: 0.6392
Epoch 4/10
94/94 [==============================] - 13s 143ms/step - loss: 0.8066 - accuracy: 0.6401
Epoch 5/10
94/94 [==============================] - 13s 139ms/step - loss: 0.8073 - accuracy: 0.6402
Epoch 6/10
94/94 [==============================] - 13s 143ms/step - loss: 0.8039 - accuracy: 0.6406
Epoch 7/10
94/94 [==============================] - 13s 143ms/step - loss: 0.8010 - accuracy: 0.6406
Epoch 8/10
94/94 [==============================] - 14s 148ms/step - loss: 0.8012 - accuracy: 0.6409
Epoch 9/10
94/94 [==============================] - 15s 157ms/step - loss: 0.7999 - accuracy: 0.6402
Epoch 10/10
47/47 [==============================] - 2s 37ms/step
train set의 accuracy : 0.642593833780

- CONVERT(sleep_rmssd_5min USING utf8) data

In [74]:
model = KerasClassifier(build_fn = stacked_bidirectional_lstm, epochs = 10, batch_size = 64, verbose = 1)
model.fit(X5_train, y5_train)
pred_train = model.predict(X5_train)
pred_val = model.predict(X5_val)
print("train set의 accuracy : {}, validation set의 예측결과 : {}" .format(accuracy_score(y5_train, pred_train), accuracy_score(y5_val, pred_val)))

pred_test = model.predict(X5_test)
print("최종 평가(test set): {}".format(accuracy_score(y5_test, pred_test)))

C:\Users\yeada\AppData\Local\Temp/ipykernel_12704/767467617.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = stacked_bidirectional_lstm, epochs = 10, batch_size = 64, verbose = 1)


Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_11 (Bidirecti  (None, 180, 40)          3520      
 onal)                                                           
                                                                 
 bidirectional_12 (Bidirecti  (None, 180, 40)          9760      
 onal)                                                           
                                                                 
 bidirectional_13 (Bidirecti  (None, 40)               9760      
 onal)                                                           
                                                                 
 dense_21 (Dense)            (None, 3)                 123       
                                                                 
 activation_21 (Activation)  (None, 3)                 0         
                                                     

C:\Users\yeada\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


94/94 [==============================] - 27s 179ms/step - loss: 0.8448 - accuracy: 0.6265
Epoch 2/10
94/94 [==============================] - 16s 168ms/step - loss: 0.8098 - accuracy: 0.6401
Epoch 3/10
94/94 [==============================] - 16s 168ms/step - loss: 0.8073 - accuracy: 0.6402
Epoch 4/10
94/94 [==============================] - 16s 174ms/step - loss: 0.8042 - accuracy: 0.6399
Epoch 5/10
94/94 [==============================] - 17s 182ms/step - loss: 0.8020 - accuracy: 0.6404
Epoch 6/10
94/94 [==============================] - 17s 176ms/step - loss: 0.8029 - accuracy: 0.6394
Epoch 7/10
94/94 [==============================] - 16s 170ms/step - loss: 0.8002 - accuracy: 0.6408
Epoch 8/10
94/94 [==============================] - 16s 167ms/step - loss: 0.7982 - accuracy: 0.6396
Epoch 9/10
94/94 [==============================] - 17s 179ms/step - loss: 0.7971 - accuracy: 0.6409
Epoch 10/10
47/47 [==============================] - 2s 39ms/step
train set의 accuracy : 0.641588471849